In [1]:

import sys
sys.path.insert(0, '../Triangulation/')
sys.path.insert(0, '../network_datasets')
sys.path.insert(0, '../Statistics/')


In [2]:
from Triangulation import *

import torch
import torch.optim as optim
import struct



import torch.nn as nn

from torch.autograd import Variable
from math import atan2,pow,acos
from  Neural_network import *

from torch.autograd.function import Function
from point_coordinates_regression import *



import matplotlib.cm as cm
import matplotlib.colors as colors
from functools import reduce



import scipy
from scipy import interpolate
from scipy.interpolate import griddata
from scipy.optimize import fmin,minimize


from polygon_population_statistics import *


%matplotlib qt

In [3]:
def get_grid_qualities(XX,YY,inner_points):
    qualities=np.empty([len(XX[0,:]),len(YY[:,0])])
    for index_x,x in enumerate(XX[0,:]):
        for index_y,y in enumerate(YY[:,0]):
            quality=np.min(np.array([np.linalg.norm(point-np.array([x,y]),2) for point in inner_points]))
            qualities[index_x][index_y]=quality
    return qualities

def get_grid_qualities_with_penalty(XX,YY,inner_points,contour):
    qualities=np.empty([len(XX[0,:]),len(YY[:,0])])
    for index_x,x in enumerate(XX[0,:]):
        for index_y,y in enumerate(YY[:,0]):
            quality=np.min(np.array([np.linalg.norm(point-np.array([x,y]),2) for point in inner_points]))
            for contour_point in contour:
                if np.linalg.norm(np.array([x,y])-contour_point)<0.2:
                    quality=2
                
            qualities[index_x][index_y]=quality
    return qualities


def get_grid_qualities_with_penalty_midpoint_included(XX,YY,inner_points,contour):
    qualities=np.empty([len(XX[0,:]),len(YY[:,0])])
    for index_x,x in enumerate(XX[0,:]):
        for index_y,y in enumerate(YY[:,0]):
            quality=np.min(np.array([np.linalg.norm(point-np.array([x,y]),2) for point in inner_points]))
            for index,contour_point in enumerate(contour):
                if np.linalg.norm(np.array([x,y])-contour_point)<0.13:
                    quality=1.3
                mid_point=0.5*(contour[index]+contour[(index+1)%len(contour)])
                if np.linalg.norm(np.array([x,y])-mid_point)<0.13:
                    quality=1.3


                
            qualities[index_x][index_y]=quality
    return qualities


def plot_grid_qualities(contour,grid_qualities,grid_points,inner_points):
    plt.clf()
    B=list(grid_qualities.flatten())
    cs = plt.scatter(grid_points[:,0],grid_points[:,1],c=B,cmap=cm.RdYlGn_r,vmin=min(grid_qualities.flatten()),vmax=max(grid_qualities.flatten()),s=4)
    plot_contour(contour)
    plot_contour(contour)
    plt.scatter(inner_points[:,0],inner_points[:,1],marker='o',c='b',label='Point location')
    plt.colorbar(cs)
    plt.legend()
    plt.show()
    

def seperate_to_sectors(grid_points,nb_sectors):
    grid_step_size=int(nb_of_grid_points/nb_sectors)
    sectors=[] 
    indices=[]
    for q in range(nb_sectors):
        for k in range(nb_sectors):
            sector_points=[]
            for j in range(grid_step_size):
                for i in range(grid_step_size):
                    index=(grid_step_size*k+i+grid_step_size*nb_sectors*j)+(grid_step_size**2)*nb_sectors*q
                    sector_points.append(grid_points[index])
                    indices.append(index)
            sectors.append(sector_points) 
    return np.array(sectors),np.array(indices)


def get_qualities_by_sector(grid_points,inner_points,contour,nb_sectors):
    grid_step_size=int(nb_of_grid_points/nb_sectors)
    quality_sectors=[] 
    for q in range(nb_sectors):
        for k in range(nb_sectors):
            quality_point=[]
            for j in range(grid_step_size):
                for i in range(grid_step_size):
                    index=(grid_step_size*k+i+grid_step_size*nb_sectors*j)+(grid_step_size**2)*nb_sectors*q
                    grid_point=grid_points[index]
                    quality=np.min(np.array([np.linalg.norm(point-grid_point,2) for point in inner_points]))
                    for index,contour_point in enumerate(contour):
                        if np.linalg.norm(grid_point-contour_point)<0.13:
                            quality=1.3
                        mid_point=0.5*(contour[index]+contour[(index+1)%len(contour)])
                        if np.linalg.norm(grid_point-mid_point)<0.13:
                            quality=1.3
                    quality_point.append(quality)
            quality_sectors.append(quality_point) 
    return np.array(quality_sectors)
    
    

def plot_grid_qualities(contour,grid_qualities,grid_points,inner_points):
    B=list(grid_qualities.flatten())
    cs = plt.scatter(grid_points[:,0],grid_points[:,1],c=B,cmap=cm.RdYlGn_r,vmin=min(grid_qualities.flatten()),vmax=max(grid_qualities.flatten()),s=4)
    plot_contour(contour)
    plot_contour(contour)
    plt.scatter(inner_points[:,0],inner_points[:,1],marker='o',c='b',label='Point location')
    plt.colorbar(cs)
  #  plt.legend()
   # plt.show()
    
    
def batch_size_factor(n,minimum,maximum):    
    factor_set=set(reduce(list.__add__, 
                ([i, n//i] for i in range(1, int(n**0.5) + 1) if n % i == 0)))
    for factor in factor_set:
        if factor>minimum and factor<maximum:
            div_factor=factor
        
    return div_factor




def save_grid_sector_qualities(filename,qualities):
    path=os.path.join('../polygon_datasets/polygon_sector_qualities/',filename)

    with open(path,'wb') as output:
        pickle.dump(qualities,output)
        
def load_grid_sector_qualities(filename):
    path=os.path.join('../polygon_datasets/polygon_sector_qualities/',filename)
        
    with open(path,'rb') as input:
        grid_qualities=pickle.load(input)        
    return grid_qualities
    
    
    

def save_grid_patch_NN(filename,net):
    path=os.path.join('../network_datasets/grid_patch_NN',filename)

    with open(path,'wb') as output:
        pickle.dump(net,output)
        
def load_grid_patch_NN(filename):
    path=os.path.join('../network_datasets/grid_patch_NN',filename)
        
    with open(path,'rb') as input:
        net=pickle.load(input)        
    net.eval()
    return net

In [4]:
def select_points(contour,grid_points,grid_qualities,nb_of_points,ring=1):
    
    selected_points=[]
    
    grid_qualities_duplicate=grid_qualities.flatten()
    label_added=False
    for i in range(nb_of_points):
        
        minimum_index=np.argmin(grid_qualities_duplicate)
        surrounding_points_index=np.array([minimum_index+1,minimum_index-1,minimum_index+nb_of_grid_points
                        ,minimum_index-nb_of_grid_points,minimum_index-nb_of_grid_points+1,minimum_index-nb_of_grid_points-1,
                        minimum_index+nb_of_grid_points+1,minimum_index+nb_of_grid_points-1])

    
        surrounding_points_index_2_ring=np.array([minimum_index+2,minimum_index-2,
                        minimum_index+2*nb_of_grid_points,
                        minimum_index+2*nb_of_grid_points+2,minimum_index+2*nb_of_grid_points-2,
                        minimum_index+2*nb_of_grid_points+1,minimum_index+2*nb_of_grid_points-1,                      
                                              
                                              
                        minimum_index-2*nb_of_grid_points,
                        minimum_index-2*nb_of_grid_points+1,minimum_index-2*nb_of_grid_points-1, 
                        minimum_index-2*nb_of_grid_points+2, minimum_index-2*nb_of_grid_points-2 ,
                                              
                       
                        minimum_index+nb_of_grid_points+2,
                        minimum_index+nb_of_grid_points-2,
                                              
                        minimum_index-nb_of_grid_points+2,
                        minimum_index-nb_of_grid_points-2


                                                                
                ])
        surrounding_points_index_3_ring=np.array([
                        minimum_index+3,minimum_index-3,
                        
                        minimum_index+3*nb_of_grid_points,
                        minimum_index+3*nb_of_grid_points+3,minimum_index+3*nb_of_grid_points-3,
                        minimum_index+3*nb_of_grid_points+2,minimum_index+3*nb_of_grid_points-2,
                        minimum_index+3*nb_of_grid_points+1,minimum_index+3*nb_of_grid_points-1, 
                        
                                              
                                              
                        minimum_index-3*nb_of_grid_points,
                        minimum_index-3*nb_of_grid_points+3, minimum_index-3*nb_of_grid_points-3 ,
                        minimum_index-3*nb_of_grid_points+2, minimum_index-3*nb_of_grid_points-2 ,
                        minimum_index-3*nb_of_grid_points+1,minimum_index-3*nb_of_grid_points-1, 
                       
                                              
                       
                        minimum_index+nb_of_grid_points+3,
                        minimum_index+nb_of_grid_points-3,
                                              
                        minimum_index-nb_of_grid_points+3,
                        minimum_index-nb_of_grid_points-3,
                
                         minimum_index+2*nb_of_grid_points+3,
                        minimum_index+2*nb_of_grid_points-3,
                                              
                        minimum_index-2*nb_of_grid_points+3,
                        minimum_index-2*nb_of_grid_points-3



                                                                
                ])
        
        
        
        try:
            surrounding_points=grid_points[np.asarray(surrounding_points_index)]
            surrounding_points_2_ring=grid_points[np.asarray(surrounding_points_index_2_ring)]
            surrounding_points_3_ring=grid_points[np.asarray(surrounding_points_index_3_ring)]
        except IndexError as e:
            print(e)
        
        point_maximum=grid_points[minimum_index]
        #if not label_added:
         #   plt.scatter(point_maximum[0],point_maximum[1],marker='d',s=30,c='y',label='Predictions')
          #  label_added=True
        #plt.scatter(point_maximum[0],point_maximum[1],marker='d',s=30,c='y')
        selected_points.append(point_maximum)
        grid_qualities_duplicate[minimum_index]=100        
        if ring==3:
            grid_qualities_duplicate[np.asarray(surrounding_points_index)]=100
            grid_qualities_duplicate[np.asarray(surrounding_points_index_2_ring)]=100
            grid_qualities_duplicate[np.asarray(surrounding_points_index_3_ring)]=100
            surrounding_points_index=np.append(surrounding_points_index_2_ring,np.append(surrounding_points_index,minimum_index))
            surrounding_points_index=np.append(surrounding_points_index,surrounding_points_index_3_ring)
            return np.array(selected_points),np.append(surrounding_points_3_ring,(np.append(surrounding_points_2_ring,np.append(surrounding_points,point_maximum)))),grid_qualities.flatten()[np.asarray(surrounding_points_index)]            
        if ring==2:
            grid_qualities_duplicate[np.asarray(surrounding_points_index)]=100
            grid_qualities_duplicate[np.asarray(surrounding_points_index_2_ring)]=100
            surrounding_points_index=np.append(surrounding_points_index_2_ring,np.append(surrounding_points_index,minimum_index))
            return np.array(selected_points),np.append(surrounding_points_2_ring,np.append(surrounding_points,point_maximum)),grid_qualities.flatten()[np.asarray(surrounding_points_index)]
        else:
            grid_qualities_duplicate[np.asarray(surrounding_points_index)]=100
            surrounding_points_index=np.append(surrounding_points_index,minimum_index)
            return np.array(selected_points),np.append(surrounding_points,point_maximum),grid_qualities.flatten()[np.asarray(surrounding_points_index)]
        
        
        
def bilineaire_interpolation(surrounding_points,grid_qualities_surrounding,selected_point):
    size=int(int(len(surrounding_points))/2)
    surrounding_points=surrounding_points.reshape(size,2)
    
 
    
    z= grid_qualities_surrounding.reshape(int(sqrt(size)),int(sqrt(size)))
    if size==9:
        z_interp = quadratic_bspline(surrounding_points[:,0].reshape(int(sqrt(size)),int(sqrt(size))),surrounding_points[:,1].reshape(int(sqrt(size)),int(sqrt(size))),z, kind='quadratic')
    elif size==25:
        z_interp = interpolate.interp2d(surrounding_points[:,0].reshape(int(sqrt(size)),int(sqrt(size))),surrounding_points[:,1].reshape(int(sqrt(size)),int(sqrt(size))),z, kind='cubic')
    else :
        z_interp = interpolate.interp2d(surrounding_points[:,0].reshape(int(sqrt(size)),int(sqrt(size))),surrounding_points[:,1].reshape(int(sqrt(size)),int(sqrt(size))),z, kind='quintic')
    x_new=np.linspace(min(surrounding_points[:,0]),max(surrounding_points[:,0]),100)
    y_new=np.linspace(min(surrounding_points[:,1]),max(surrounding_points[:,1]),100)
    z_new=z_interp(x_new,y_new)
    epsilon=1e-4
    bnds=((min(surrounding_points[:,0]),max(surrounding_points[:,0])),(min(surrounding_points[:,1]),max(surrounding_points[:,1])))
    minimum=minimize(lambda v: z_interp(v[0],v[1]), np.array([selected_point[:,0]+epsilon,selected_point[:,1]+epsilon]), method='TNC',bounds=bnds)
    return np.array([minimum.x[0],minimum.x[1]])




class quadratic_bspline(interpolate.interp2d):
    ''' Iveride interp2d to include quadratic spline'''
    def __init__(self,*args,**kws):
        try:
            super(quadratic_bspline,self).__init__(*args,**kws)
        
        except ValueError:      
            kx=ky=2
            x=args[0]
            y=args[1]
            z=args[2]
            rectangular_grid = (z.size == len(x) * len(y))
            if rectangular_grid:
                self.tck = scipy.interpolate.fitpack.bisplrep(x, y, z, kx=kx, ky=ky, s=0.0)
            else:
                nx, tx, ny, ty, c, fp, ier = scipy.interpolate.dfitpack.regrid_smth(
                x, y, z, None, None, None, None,
                kx=kx, ky=ky, s=0.0)
                self.tck = (tx[:nx], ty[:ny], c[:(nx - kx - 1) * (ny - ky - 1)],
                        kx, ky)
            self.bounds_error = False
            self.fill_value = None
            self.x, self.y, self.z = [np.array(a, copy=copy) for a in (x, y, z)]

            self.x_min, self.x_max = np.amin(x), np.amax(x)
            self.y_min, self.y_max = np.amin(y), np.amax(y)
            
def replace_target_edge_lengths(polygons_reshaped_with_sector_grid_points,nb_of_edges,median_edge_legths):
    target_edge_lengths=np.round(polygons_reshaped_with_sector_grid_points[:,2*nb_of_edges],1)
    for index,target_edge_length in  enumerate(target_edge_lengths):
        polygons_reshaped_with_sector_grid_points[index][2*nb_of_edges] =median_edge_legths[str(target_edge_length)]
    return polygons_reshaped_with_sector_grid_points

In [315]:
nb_of_edges=8
nb_of_points=1

In [316]:
directory_path='../polygon_datasets/additional_polygon_datasets/'+str(nb_of_edges)'_polygons/'

with open(os.path.join(directory_path,str(nb_of_edges)+'_'+str(nb_of_points)+'_polygons.pkl'),'rb') as f:
    polygons=pickle.load(f)


In [317]:
polygon_list=[]
points_list=[]
target_edge_length_list=[]
for polygon in polygons:
    for target_edge_length in np.linspace(0.1,1,10):
        nb,point=get_extrapoints_target_length_jupyter(polygon,target_edge_length,algorithm='del2d')
        if nb==1:
            polygon_list.append(polygon)
            target_edge_length_list.append(target_edge_length)
            points_list.append(point)
        

9 0.01994567341046799 0.1555681078063858 0
10 0.3597475977166628 0.5419401457550153 0
11 -0.1008361980610126 -0.2200365047286581 0
12 -0.4479137353873963 -0.3795593106551434 0
13 -0.2084180531190768 0.007654570438783052 0
14 -0.3867922603384394 0.3266459321572841 0
15 0.0726899775265869 0.4581505233606726 0
16 0.2414731544615374 0.2765860448514111 0
17 -0.1564954990233023 0.412355381180144 0
18 -0.332068113155683 0.5077651019842809 0
19 -0.2223030367053125 0.2035813351915013 0
9 -0.02555847093892465 0.2222926724901892 0
10 0.3201748249818039 0.4266526770340386 0
11 -0.009987719067695505 -0.1634541882942831 0
9 -0.01526903582694779 0.2133296093887049 0
9 -0.01526903582694779 0.2133296093887049 0
9 -0.01526903582694779 0.213329609388705 0
9 -0.01526903582694779 0.2133296093887049 0
9 -0.01526903582694779 0.2133296093887049 0
9 -0.01526903582694779 0.2133296093887049 0
9 -0.01526903582694779 0.2133296093887049 0
9 -0.01526903582694779 0.2133296093887049 0
9 0.258539191066683 -0.4590221757

9 -0.3058428404471695 -0.3085596102190485 0
9 -0.3058428404471695 -0.3085596102190485 0
9 -0.3058428404471695 -0.3085596102190485 0
9 -0.3058428404471695 -0.3085596102190485 0
9 -0.3058428404471695 -0.3085596102190485 0
9 -0.3058428404471696 -0.3085596102190485 0
9 -0.3058428404471695 -0.3085596102190485 0
9 -0.06116862483529659 -0.2987406000100043 0
10 0.1797441524901451 0.1014380854706998 0
11 -0.2625455231073133 0.07483553647202362 0
12 -0.5464368696318411 -0.2591433708383202 0
13 0.4084052706036149 -0.1820043981919703 0
14 0.1914353738550931 -0.6517892547087824 0
15 -0.2977143817547656 -0.6952272942579401 0
16 -0.3092345551879726 -0.1692708543510171 0
17 -0.4792633800333794 0.0137685670694565 0
18 -0.05084086732401774 -0.6086546448230754 0
19 -0.05350035299583079 -0.03172600709497388 0
20 0.6119186218884342 0.008298988385009849 0
21 0.3721661650863058 0.0329842458931774 0
22 -0.04557328204849567 0.2284926488324542 0
23 0.2214152836335646 -0.3845162846572392 0
24 -0.3525706729469683

9 -0.1403477712081206 0.2101417619448983 0
9 -0.1403477712081206 0.2101417619448983 0
9 -0.1403477712081206 0.2101417619448983 0
9 -0.1403477712081206 0.2101417619448983 0
9 -0.1403477712081206 0.2101417619448983 0
9 -0.1403477712081206 0.2101417619448983 0
9 -0.1403477712081206 0.2101417619448983 0
9 -0.1403477712081206 0.2101417619448983 0
9 -0.1524949805275925 0.06696791186420738 0
10 0.1660213484954066 0.250999107339655 0
11 0.6433688146472056 0.06655612288748239 0
12 0.4111362998032966 0.1715771407899776 0
13 -0.1402306872259328 -0.3763071235893023 0
14 -0.397127412764151 -0.1645279445836867 0
15 -0.3974564500095599 0.3689361331726841 0
16 -0.09631968383698297 0.3171198832345172 0
9 -0.1521837117548766 0.07925458220873774 0
10 0.257306821250747 0.2217360282335745 0
11 0.6049114352439116 0.07909584546477208 0
9 -0.1521837117548766 0.07925458220873774 0
9 -0.1521837117548766 0.07925458220873774 0
9 -0.1521837117548766 0.07925458220873774 0
9 -0.1521837117548766 0.07925458220873774 0

9 0.1136773913889409 -0.4987778711542376 0
10 0.2552740561993011 -0.7153499541084765 0
11 0.4354209144809212 -0.3994063049798327 0
12 0.2270077649387751 0.06315746109964879 0
13 0.2761314638165603 -0.2393681961653175 0
14 0.02226585222475655 -0.1515086100922247 0
15 -0.1693191495208723 -0.3770890459597914 0
16 0.2933480114813604 0.45523537408784 0
17 0.2679107275213901 -0.5002074744284499 0
18 -0.4333599028108273 -0.1250815624304325 0
19 -0.1845607338074451 -0.09721765780866983 0
20 0.03157178690260106 0.6772020606492619 0
21 0.2062918471135766 0.8124625563848879 0
9 0.1163542321818495 -0.3991568182784215 0
9 0.1163542321818495 -0.3991568182784215 0
9 0.1163542321818495 -0.3991568182784215 0
9 0.1163542321818495 -0.3991568182784215 0
9 0.1163542321818495 -0.3991568182784215 0
9 0.1163542321818495 -0.3991568182784215 0
9 0.1163542321818495 -0.3991568182784215 0
9 0.1163542321818495 -0.3991568182784215 0
9 0.1163542321818495 -0.3991568182784215 0
9 0.2207266165190396 0.5380461988987952 0

9 0.1306090216485352 0.1587850269491052 0
9 0.1306090216485352 0.1587850269491052 0
9 0.1306090216485352 0.1587850269491052 0
9 0.1306090216485352 0.1587850269491052 0
9 0.1306090216485352 0.1587850269491052 0
9 0.1306090216485352 0.1587850269491052 0
9 0.1306090216485352 0.1587850269491052 0
9 0.1306090216485352 0.1587850269491052 0
9 -0.1756808164045468 -0.009273393117718742 0
10 0.2753520277424969 0.1791769098659625 0
11 0.1700157568852837 -0.2811014923968605 0
12 -0.04486424960039515 0.3995627099084651 0
13 -0.4920287819419384 0.2885264680033424 0
14 -0.6510451693185952 -0.2296348233498481 0
15 -0.4623537453226556 0.02335627965428613 0
16 -0.3715420984518518 -0.4308758373935861 0
17 -0.09384718294370434 -0.2705030099947846 0
18 0.08594305057025647 -0.04593747093381913 0
19 0.3338203438981802 -0.09220587267098704 0
20 -0.242085356136871 0.2300865024508169 0
21 -0.2776600809635495 0.4768372825799265 0
22 0.009407756054777615 0.1901462796330632 0
23 0.1577940346285191 0.38723741878088

...

9 0.05774964085496118 0.1334970981789267 0
9 0.05774964085496116 0.1334970981789267 0
9 -0.1748275117773863 0.06320373938220357 -0
10 -0.1984797343258205 -0.2736186439077483 0
11 0.2201518837396636 -0.04657495572911435 0
9 -0.1867895451956128 0.02149258559080158 -0
9 -0.1867895451956128 0.02149258559080158 -0
9 -0.1867895451956128 0.02149258559080158 -0
9 -0.1867895451956128 0.02149258559080158 -0
9 -0.1867895451956128 0.02149258559080158 -0
9 -0.1867895451956128 0.02149258559080158 -0
9 -0.1867895451956128 0.02149258559080158 -0
9 -0.1867895451956128 0.02149258559080158 -0
9 -0.1867895451956128 0.02149258559080158 -0
9 0.5649724355830735 -0.1642685559837478 0
10 0.2252440412446113 -0.004879361896369986 0
11 0.3080854530144859 -0.3002490153768272 0
12 0.4104360946820893 -0.09419564679679959 0
13 0.5434324711745577 0.09262253205220404 0
9 0.5469547185765541 -0.05746468059415969 0
10 0.20087723357276 -0.09891252365170405 0
9 0.4640661527410822 -0.1278572469055952 0
9 0.4640661527410822 -

9 -0.02834227612087882 0.04438133703971726 0
9 -0.02834227612087883 0.04438133703971729 0
9 -0.02834227612087883 0.04438133703971729 0
9 -0.02834227612087883 0.04438133703971729 0
9 -0.02834227612087883 0.04438133703971728 0
9 -0.02834227612087883 0.04438133703971728 0
9 -0.02834227612087882 0.04438133703971728 0
9 -0.02834227612087883 0.04438133703971729 0
9 -0.4449162971245752 -0.2440876265809301 0
10 -0.3740159433066278 -0.7119456736728151 0
11 -0.1011489711801207 -0.6379801907656965 0
12 -0.3295390224492873 -0.4696071437843269 0
13 0.01618800914520784 -0.02124900794980695 0
14 -0.2989362028661701 -0.03309565176896796 0
15 -0.1342985971714688 -0.3013290164158302 0
16 -0.5933153034636444 -0.4526932114863628 0
17 0.255026993397892 0.07933236327722582 0
18 0.4803992310323034 0.1631541629870857 0
19 0.1237054186474809 0.1938426062944889 0
9 -0.3234565975329616 -0.3838380758320846 0
9 -0.3234565975329616 -0.3838380758320846 0
9 -0.3234565975329616 -0.3838380758320845 0
9 -0.3234565975329

9 -0.002074037271857178 -0.2907136585841264 0
9 -0.002074037271857163 -0.2907136585841264 0
9 -0.00207403727185718 -0.2907136585841264 0
9 -0.002074037271857232 -0.2907136585841265 0
9 -0.002074037271857233 -0.2907136585841264 0
9 -0.00207403727185721 -0.2907136585841265 0
9 -0.002074037271857232 -0.2907136585841266 0
9 -0.002074037271857234 -0.2907136585841265 0
9 0.2244422000859294 0.5312475131790715 0
10 0.01328069010442021 0.1881070643573022 0
11 -0.4282553661838613 -0.03371272749958689 0
12 0.1972214687276274 -0.2308150939496744 0
13 -0.1137451245966864 -0.1093982966776803 0
14 0.09062702911000418 -0.5950207773577559 0
9 0.2572128278081286 0.5904660165050324 0
9 0.2572128278081286 0.5904660165050324 0
9 0.2572128278081286 0.5904660165050324 0
9 0.2572128278081286 0.5904660165050324 0
9 0.2572128278081286 0.5904660165050324 0
9 0.2572128278081286 0.5904660165050324 0
9 0.2572128278081286 0.5904660165050324 0
9 0.2572128278081286 0.5904660165050324 0
9 0.2572128278081286 0.590466016

9 0.2805445880841231 0.06818413620917727 0
9 0.2805445880841231 0.06818413620917727 0
9 0.2805445880841231 0.06818413620917727 0
9 0.2805445880841232 0.06818413620917727 0
9 0.2805445880841231 0.06818413620917728 0
9 0.2805445880841232 0.06818413620917727 0
9 0.2805445880841231 0.06818413620917727 0
9 0.2805445880841231 0.06818413620917727 0
9 0.005729163974398427 0.1945972778844693 0
10 -0.1242319090916067 -0.339659726154233 0
11 -0.3586967716172101 -0.1163279170523841 0
12 -0.3914111860025766 0.3299434455876233 0
13 -0.02703715233956512 -0.5990126212490314 0
14 -0.002706081542831092 0.4081881019882039 0
15 -0.1747214507976189 0.4031649354169383 0
16 -0.1306724106746719 -0.05037179964664021 0
9 -0.1375011515061655 0.04933492221050682 0
9 -0.1375011515061655 0.04933492221050682 0
9 -0.1375011515061655 0.04933492221050681 0
9 -0.1375011515061655 0.04933492221050682 0
9 -0.1375011515061655 0.0493349222105068 0
9 -0.1375011515061655 0.04933492221050682 0
9 -0.1375011515061655 0.0493349222

9 0.08264061612432563 -0.08276983889549028 0
10 -0.3374204424266436 -0.3717096054376969 0
11 0.4073033263267166 -0.3929751554419825 0
9 0.1374978955233641 -0.1063843040979379 0
9 0.1374978955233641 -0.1063843040979379 0
9 0.1374978955233641 -0.1063843040979379 0
9 0.1374978955233641 -0.1063843040979379 0
9 0.1374978955233641 -0.1063843040979379 0
9 0.1374978955233641 -0.1063843040979379 0
9 0.1374978955233641 -0.1063843040979379 0
9 0.1374978955233641 -0.1063843040979379 0
9 -0.346508022859132 -0.1817322385203777 0
10 0.05347180012438186 -0.2858847510509206 0
11 -0.07214596021866188 0.01276600597687719 0
12 -0.4234688621594993 0.198063153109843 0
13 -0.1875437176508189 -0.1567573524631224 0
14 -0.3178085497469454 -0.02661355930251647 0
15 -0.5142839766010299 -0.06748712043047812 0
16 -0.6334467222250571 -0.231473172600848 0
17 -0.379185288211057 -0.5031482031227486 0
18 -0.2374842595442033 -0.3429310970515096 0
19 0.4006825545772551 -0.4412049342046214 0
20 -0.4852658891176356 0.454244

In [318]:
len(target_edge_length_list)

1704

In [319]:
target_edge_length_list

[0.30000000000000004,
 0.4,
 0.5,
 0.6,
 0.7000000000000001,
 0.8,
 0.9,
 1.0,
 0.30000000000000004,
 0.4,
 0.5,
 0.6,
 0.7000000000000001,
 0.8,
 0.9,
 1.0,
 0.2,
 0.30000000000000004,
 0.4,
 0.5,
 0.6,
 0.7000000000000001,
 0.8,
 0.9,
 1.0,
 0.2,
 0.30000000000000004,
 0.4,
 0.5,
 0.6,
 0.7000000000000001,
 0.8,
 0.9,
 1.0,
 0.30000000000000004,
 0.4,
 0.5,
 0.6,
 0.7000000000000001,
 0.8,
 0.9,
 1.0,
 0.2,
 0.30000000000000004,
 0.4,
 0.5,
 0.6,
 0.7000000000000001,
 0.8,
 0.9,
 1.0,
 0.30000000000000004,
 0.4,
 0.5,
 0.6,
 0.7000000000000001,
 0.8,
 0.9,
 1.0,
 0.30000000000000004,
 0.4,
 0.5,
 0.6,
 0.7000000000000001,
 0.8,
 0.9,
 1.0,
 0.30000000000000004,
 0.4,
 0.5,
 0.6,
 0.7000000000000001,
 0.8,
 0.9,
 1.0,
 0.30000000000000004,
 0.4,
 0.5,
 0.6,
 0.7000000000000001,
 0.8,
 0.9,
 1.0,
 0.2,
 0.30000000000000004,
 0.4,
 0.5,
 0.6,
 0.7000000000000001,
 0.8,
 0.9,
 1.0,
 0.2,
 0.30000000000000004,
 0.4,
 0.5,
 0.6,
 0.7000000000000001,
 0.8,
 0.9,
 1.0,
 0.30000000000000004,


In [320]:
# load initial data

nb_of_grid_points=10
X=np.linspace(-1.3,1.3,nb_of_grid_points)
Y=np.linspace(-1.3,1.3,nb_of_grid_points)
XX,YY=np.meshgrid(X,Y)

grid_points=np.array([[x,y] for x in X for y in Y])

nb_of_edges=8

nb_of_points=1
nb_sectors=int(nb_of_grid_points/2)
sectors,indices=seperate_to_sectors(grid_points,nb_sectors)

polygons_qualities_sector=load_grid_sector_qualities(str(nb_of_edges)+'_'+str(nb_of_points)+'_'+str(nb_of_grid_points)+'_sector_qualities.pkl')


polygons_initial=load_dataset(str(nb_of_edges)+'_polygons.pkl')
number_of_insertion_points=load_dataset(str(nb_of_edges)+'_nb_of_points_del.pkl')
point_coordinates_initial=load_dataset(str(nb_of_edges)+'_point_coordinates_del.pkl')


polygons_reshaped,point_coordinates=reshape_data(polygons_initial,point_coordinates_initial,number_of_insertion_points,int(nb_of_points))


polygons_qualities_sector=polygons_qualities_sector.reshape(int(polygons_qualities_sector.shape[0]*polygons_qualities_sector.shape[1]),1
                                                            ,int(polygons_qualities_sector.shape[2]))

In [322]:
polygon_list=np.array(polygon_list)

In [323]:
additional_polygons_reshaped=np.hstack([polygon_list.reshape(len(polygon_list),2*nb_of_edges),np.array(target_edge_length_list).reshape(len(target_edge_length_list),1)])

In [324]:
additional_point_coordinates=np.array(points_list).reshape(len(points_list),1,2)

In [325]:
# Adding new data to the existing

polygon_reshaped_total=np.vstack([polygons_reshaped,additional_polygons_reshaped])
point_coordinates_total=np.vstack([point_coordinates,additional_point_coordinates])

In [327]:
# calculate sector qualities

grid_points_list=[10,20,40]
for nb_of_grid_points in grid_points_list:
    X=np.linspace(-1.3,1.3,nb_of_grid_points)
    Y=np.linspace(-1.3,1.3,nb_of_grid_points)
    XX,YY=np.meshgrid(X,Y)

    grid_points=np.array([[x,y] for x in X for y in Y])
    edges=[8]


    nb_sectors=int(nb_of_grid_points/2)# Grid will be divided into nb_sectors**2 patches
    for nb_of_edges in edges:
        points=[1]
        for nb_of_points in points:
            grid_step_size=int(nb_of_grid_points/nb_sectors)
            polygons_reshaped,point_coordinates=polygon_reshaped_total,point_coordinates_total
            sectors,indices=seperate_to_sectors(grid_points,nb_sectors)
            polygons_sectors=[]
            polygons_qualities_sector=[]
            for index,points in enumerate(point_coordinates):
                inner_points=points.reshape(nb_of_points,2)
                contour=np.delete(polygons_reshaped[index],2*nb_of_edges).reshape(nb_of_edges,2)    
                grid_qualities=get_qualities_by_sector(grid_points,inner_points,contour,nb_sectors)
                polygons_qualities_sector.append(grid_qualities)
            polygons_qualities_sector=np.array(polygons_qualities_sector)
            
            save_grid_sector_qualities(str(nb_of_edges)+'_'+str(nb_of_points)+'_'+str(nb_of_grid_points)+'additional_sector_qualities.pkl',polygons_qualities_sector)



In [285]:
# Calculate quality of each patch of points

nb_of_grid_points=40
X=np.linspace(-1.3,1.3,nb_of_grid_points)
Y=np.linspace(-1.3,1.3,nb_of_grid_points)
XX,YY=np.meshgrid(X,Y)

grid_points=np.array([[x,y] for x in X for y in Y])

nb_of_edges=10

nb_of_points=1
nb_sectors=int(nb_of_grid_points/2)
sectors,indices=seperate_to_sectors(grid_points,nb_sectors)

polygons_qualities_sector=load_grid_sector_qualities(str(nb_of_edges)+'_'+str(nb_of_points)+'_'+str(nb_of_grid_points)+'additional_sector_qualities.pkl')



polygons_reshaped,point_coordinates=polygon_reshaped_total,point_coordinates_total


polygons_qualities_sector=polygons_qualities_sector.reshape(int(polygons_qualities_sector.shape[0]*polygons_qualities_sector.shape[1]),1
                                                            ,int(polygons_qualities_sector.shape[2]))

In [286]:
polygons_reshaped_with_sector_grid_points=[]
for polygon in polygons_reshaped:
    for sector in sectors:
        polygons_reshaped_with_sector_grid_points.append( np.hstack([polygon,sector.reshape(2*len(sector))]))
polygons_reshaped_with_sector_grid_points=np.array(polygons_reshaped_with_sector_grid_points)


In [287]:
# 80/20 training/test data ratio

nb_of_test_data=int(len(polygons_reshaped_with_sector_grid_points)*0.2)
nb_of_training_data=int(len(polygons_reshaped_with_sector_grid_points)-nb_of_test_data)
nb_of_test_data,nb_of_training_data


(149120, 596480)

In [288]:
                  ################## Training with median edge length ##############
#median_edge_legths=get_median_target_edge_length(10)
#polygons_reshaped_with_sector_grid_points=replace_target_edge_lengths(polygons_reshaped_with_sector_grid_points,nb_of_edges,median_edge_legths)

In [289]:
# Shuffle the data
polygons_reshaped_with_sector_grid_points,polygons_qualities_sector=unison_shuffled_copies(polygons_reshaped_with_sector_grid_points,polygons_qualities_sector)


In [290]:
# Setting up the variables

x_tensor=torch.from_numpy(polygons_reshaped_with_sector_grid_points[:nb_of_training_data]).type(torch.FloatTensor)
x_tensor_test=torch.from_numpy(polygons_reshaped_with_sector_grid_points[nb_of_training_data:]).type(torch.FloatTensor)
x_variable,x_variable_test=Variable(x_tensor),Variable(x_tensor_test)

y_tensor=torch.from_numpy(polygons_qualities_sector[:nb_of_training_data]).type(torch.FloatTensor)
y_tensor_test=torch.from_numpy(polygons_qualities_sector[nb_of_training_data:]).type(torch.FloatTensor)

y_variable,y_variable_test=Variable(y_tensor),Variable(y_tensor_test)



In [291]:
my_net=Net(x_variable.size()[1],y_variable.size()[2],nb_of_hidden_layers=2, nb_of_hidden_nodes=80,batch_normalization=True)
torch.cuda.empty_cache()
print("Training data length:",x_variable_test.size()[1],y_variable.size()[2])
x_variable.size()

Training data length: 29 4


torch.Size([596480, 29])

In [292]:
optimizer = torch.optim.Adam(my_net.parameters(), lr=1e-4,weight_decay=0.2)
loss_func =torch.nn.MSELoss(size_average=False) 

In [293]:

if  torch.cuda.is_available():
    loss_func.cuda()
        
    x_variable , y_variable=x_variable.cuda(), y_variable.cuda()
    x_variable_test,y_variable_test= Variable(x_tensor_test.cuda(),volatile=True),Variable(y_tensor_test.cuda(),volatile=True)

    print("cuda activated")
    

cuda activated


In [294]:
training_data_size=int(x_variable.size()[0])
print("Training data size: ",training_data_size)
batch_size_div=batch_size_factor(training_data_size,100,500)
batch_size=int(training_data_size/batch_size_div)
print("Batch size: ", batch_size)
nb_of_epochs=300
my_net.cuda()
#my_net.cpu()

# Train the network #
my_net.train()
for t in range(nb_of_epochs):
    sum_loss=0
    for b in range(0,x_variable.size(0),batch_size):
        out = my_net(x_variable.narrow(0,b,batch_size))                 # input x and predict based on x
        loss= loss_func(out, y_variable.narrow(0,b,batch_size))     # must be (1. nn output, 2. target), the target label is NOT one-hotted
        
        sum_loss+=float(loss.data[0])

        
        optimizer.zero_grad()   # clear gradients for next train
        loss.backward()         # backpropagation, compute gradients
        optimizer.step()        # apply gradients
        del loss
        del out
    if t%10==0: 
        my_net.eval()
        out_test=my_net(x_variable_test)   
        test_loss=loss_func(out_test,y_variable_test)
        print("Epoch:",t,"Training Loss:",sum_loss/(x_variable.size(0)),test_loss.data[0]/(x_variable_test.size(0)))
        my_net.train()


Training data size:  596480
Batch size:  2560
Epoch: 0 Training Loss: 6.128480488854928 3.740444356893777
Epoch: 10 Training Loss: 0.17878624855704572 0.1763688378067999
Epoch: 20 Training Loss: 0.14898276850900938 0.14773170569423955
Epoch: 30 Training Loss: 0.13512793723094105 0.13412516188723847
Epoch: 40 Training Loss: 0.12804597150614333 0.127137972459261
Epoch: 50 Training Loss: 0.12418816504048687 0.12334703437248525
Epoch: 60 Training Loss: 0.12185143396066493 0.12109168056766363
Epoch: 70 Training Loss: 0.12014434056220648 0.1194194613608168
Epoch: 80 Training Loss: 0.11876655663543505 0.11810525394816256
Epoch: 90 Training Loss: 0.11739482321964313 0.11672663217962044
Epoch: 100 Training Loss: 0.11582677507605164 0.1151604386358302
Epoch: 110 Training Loss: 0.11459629018930918 0.11394743694256304
Epoch: 120 Training Loss: 0.11362987250217553 0.11295987239723042
Epoch: 130 Training Loss: 0.11282552194186035 0.11215025283747988
Epoch: 140 Training Loss: 0.11207866180096573 0.11

In [295]:
save_grid_patch_NN(str(nb_of_edges)+'_'+str(nb_of_points)+'_'+str(nb_of_grid_points)+'_additional_grid_NN.pkl',my_net)

In [296]:
#  Statistical convergence

nb_of_edges=10
nb_of_points=1
nb_of_grid_points_list=[10,20,40]
errors_in_grid_resolution=[]
errors_in_grid_resolution_with_interpolation=[]

with open('../polygon_datasets/validation_sets/grid_patch_regression/'+str(nb_of_edges)+'_'+str(nb_of_points)+'_valdation_polygon_data.pkl','rb') as f:
    contours=pickle.load(f)

for random_contour in contours:
    random_nb_of_points,random_point_coordinated_delaunay=get_extrapoints_target_length_jupyter(random_contour,1,algorithm='del2d')  
    random_contour_reshaped=random_contour.reshape(1,2*nb_of_edges)
    #random_contour_with_target=np.hstack([random_contour_reshaped,[[1]]])
    random_contour_with_target=np.hstack([random_contour_reshaped,[median_edge_legths['1.0'].reshape(1)]])
    
    predicted_points_grid_resolution=[]
    errors=[]
    errors_with_interpolation=[]
    predicted_points=[]
    for nb_of_grid_points in nb_of_grid_points_list:

        X=np.linspace(-1.3,1.3,nb_of_grid_points)
        Y=np.linspace(-1.3,1.3,nb_of_grid_points)
        XX,YY=np.meshgrid(X,Y)
        grid_points=np.array([[x,y] for x in X for y in Y])
    

        nb_sectors=int(nb_of_grid_points/2)
        sectors,indices=seperate_to_sectors(grid_points,nb_sectors)







        grid_step_size=int(nb_of_grid_points/nb_sectors)

        random_point_coordinated_delaunay=np.array(random_point_coordinated_delaunay)
        random_point_coordinated_delaunay.reshape(nb_of_points,2)
        my_net=load_grid_patch_NN(str(nb_of_edges)+'_'+str(nb_of_points)+'_'+str(nb_of_grid_points)+'_additional_grid_NN.pkl')
        my_net=my_net.cpu()
        my_net.eval()
        random_grid_qualities=[]
        random_sector_qualities=[]

        for sector in sectors:
            random_contour_with_sector_points=np.hstack([random_contour_with_target,sector.reshape(1,2*len(sector))])
            random_contour_with_sector_points=Variable(torch.from_numpy(random_contour_with_sector_points))
            random_contour_with_sector_points=random_contour_with_sector_points.expand(1000,random_contour_with_sector_points.shape[1]).type(torch.FloatTensor)
            random_sector_quality=my_net(random_contour_with_sector_points)
            random_sector_quality=random_sector_quality.data[0]
            random_sector_quality=random_sector_quality.numpy()
            random_sector_qualities.append(random_sector_quality)
        random_sector_qualities=np.array(random_sector_qualities)

        grid_qualities=np.empty((grid_step_size**2)*(nb_sectors**2))
        for index,point_index in enumerate(indices):
            grid_qualities[point_index]=random_sector_qualities.flatten()[index]

    
        predicted_point,surrounding_points,grid_qualities_surrounding=select_points(random_contour,grid_points,grid_qualities,nb_of_points=1,ring=1)
        interpolated_point=bilineaire_interpolation(surrounding_points,grid_qualities_surrounding,predicted_point)
        error=np.linalg.norm(random_point_coordinated_delaunay.reshape(2)-predicted_point)
        error_with_interpolation=np.linalg.norm(random_point_coordinated_delaunay.reshape(2)-interpolated_point)
        print(error)
        predicted_points.append(predicted_point)

        predicted_points_grid_resolution.append(predicted_point)
        errors.append(error)
        errors_with_interpolation.append(error_with_interpolation)
    errors_in_grid_resolution.append(errors)
    errors_in_grid_resolution_with_interpolation.append(errors_with_interpolation)

11 -0.07773575816372752 0.003668377205892672 0
0.15578173812245258
0.15985844462098112
0.18154648526717246
11 -0.3574843338093715 -0.6165514376082681 0
0.7902535839601151
0.8864942480381037
0.6416293500192938
11 -0.0551597755170004 0.03654949628308437 -0
0.22689901821109656
0.10580490352604348
0.1315955798994055
11 0.04743876531221136 -0.03447790910816691 0
0.14663812832197004
0.03990480388879817
0.06702318707841642
11 -0.06033574411241774 0.06911362276797743 -0
0.11291149704965546
0.29909624485216696
0.19332178737857744
11 -0.0519964694359893 0.0881100850274818 0
0.1082598177260773
0.12201654649095833
0.16156552617179196
11 0.1065769481340494 -0.0006473960144732218 0
0.35752115662749095
0.12045504802748094
0.17777730407506329
11 -0.02871259585143025 0.06913049140073888 -0
0.18882677809859683
0.03971479375784448
0.0776841074570686
11 -0.02823369808016717 -0.04035539479308997 0
0.15601111443696725
0.10064701677176985
0.1404480086268205
11 -0.01902564629091782 -0.06893638121740087 0
0.26

0.13120209366968436
0.28367882901682356
0.12236200543621059
11 0.08591565236806939 -0.1735196097455521 0
0.06535277147303166
0.03624519097810935
0.0903882364702899
11 -0.06159656639545862 0.08272582442490968 -0
0.2418059179494874
0.1513008660131074
0.14993803374084655
11 0.06364233547563668 -0.01742715695026552 0
0.2437898670984648
0.05121731674358949
0.05912074918921002
11 0.1507492241228738 -0.00352451649152204 0
0.32710507189049903
0.08475452423476151
0.08779920646334874
11 0.04657298249940622 -0.06437570959620148 0
0.20711991622354314
0.022219427732824593
0.06421476672684343
11 -0.006781678210243976 -0.3429926419977565 0
0.24958110290011934
0.14927524950870727
0.10265487218464485
11 0.09287392189835847 0.08823588029951428 0
0.0762818555025448
0.22484986867683557
0.13536674403256727
11 -0.101168158976761 0.02270632077181692 0
0.12920142274453641
0.056233528103726546
0.08798867201122604
11 -0.1709341264688984 -0.0289362825686668 0
0.11850670324364576
0.10985434617936951
0.07107027756

In [297]:
errors_in_grid_resolution_with_interpolation=np.array(errors_in_grid_resolution_with_interpolation)
errors_in_grid_resolution_with_interpolation_10=errors_in_grid_resolution_with_interpolation


In [300]:
errors_in_grid_resolution=np.array(errors_in_grid_resolution)
errors_in_grid_resolution_with_interpolation=np.array(errors_in_grid_resolution_with_interpolation)
with open('../polygon_datasets/additional_polygon_datasets/10_polygons/10_1_additional_mean_error_with_median.pkl','wb') as f:
    pickle.dump(errors_in_grid_resolution,f)
with open('../polygon_datasets/additional_polygon_datasets/10_polygons/10_1_additional_mean_error_with_interpolation_with_median.pkl','wb') as f:
    pickle.dump(errors_in_grid_resolution_with_interpolation,f)

x_label=[10,20,40]
y_label=[]
y_label_with_interpolation=[]
for i in range(3):
    y_label.append( errors_in_grid_resolution[:,i].mean())
for i in range(3):
    y_label_with_interpolation.append( errors_in_grid_resolution_with_interpolation[:,i].mean())
plt.plot(x_label,y_label_with_interpolation,'--b')
plt.plot(x_label,y_label,'b')
plt.xlabel('resolution')
plt.ylabel('mean error')

Text(0, 0.5, 'mean error')

In [309]:
file='../polygon_datasets/additional_polygon_datasets/10_polygons/10_1_additional_mean_error_with_median.pkl'
with open(file,'rb') as f:
    errors_in_grid_resolution=pickle.load(f)
    
    
file='../polygon_datasets/additional_polygon_datasets/10_polygons/10_1_additional_mean_error_with_interpolation_with_median.pkl'
with open(file,'rb') as f:
    errors_in_grid_resolution_with_interpolation=pickle.load(f)

In [310]:
x_label=[10,20,40]
y_label=[]
y_label_with_interpolation=[]
for i in range(3):
    y_label.append( errors_in_grid_resolution[:,i].mean())
for i in range(3):
    y_label_with_interpolation.append(errors_in_grid_resolution_with_interpolation[:,i].mean())
plt.plot(x_label,y_label_with_interpolation,'--b')
plt.plot(x_label,y_label,'b')
plt.xlabel('resolution')
plt.ylabel('mean error')

Text(33.847222222222214, 0.5, 'mean error')

In [184]:
    ######################## Train by replacing target edge length with  median edge length #################################

In [226]:
median_edge_legths=get_median_target_edge_length(nb_of_edges)


In [230]:
for index,target_edge_length in  enumerate(target_edge_lengths):
    polygons_reshaped_with_sector_grid_points[index][2*nb_of_edges] =median_edge_legths[str(target_edge_length)]

In [232]:
polygons_reshaped_with_sector_grid_points[:,20]

array([0.42829299, 0.42829299, 0.42829299, ..., 0.42829299, 0.42829299,
       0.42762375])

In [276]:
median_edge_legths

{'0.1': 0.2885622452262092,
 '0.2': 0.42762375498369876,
 '0.3': 0.4282929922730428,
 '0.4': 0.42829299227304285,
 '0.5': 0.42829299227304285,
 '0.6': 0.42829299227304285,
 '0.7': 0.42829299227304285,
 '0.8': 0.42829299227304285,
 '0.9': 0.42829299227304285,
 '1.0': 0.42829299227304285}

In [278]:
median_edge_legths['1.0']

0.42829299227304285